In [104]:
%matplotlib inline

import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats

from tqdm import tqdm_notebook as tqdm
from collections import OrderedDict

import itertools
import sys

sys.path.append("../..")
from networkentropy import network_energy as ne, network_energy_gradient as neg, network_utils as nu

## Correlaction between energy gradient and length of the shortest path for all node pairs
### For sythetic networks

#### Define constants

In [2]:
GENERATORS = ['random', 'smallworld', 'waxman', 'powerlaw']
RANDOM, SMALLWORLD, WAXMAN, POWERLAW = GENERATORS

METHODS = ['randic', 'laplacian', 'graph']
RANDIC, LAPLACIAN, GRAPH = METHODS

PARAMETER_MAX = 4
PARAMETERS = list(range(1, PARAMETER_MAX)) #parameters for synthetic networks models

#### Create synthetic networks

In [3]:
def create_graph(p, generator, p_max, num_nodes=100):
    if generator == RANDOM:
        return nx.erdos_renyi_graph(n=num_nodes, p=p/(p_max*10))
    elif generator == SMALLWORLD:
        return nx.watts_strogatz_graph(n=num_nodes, k=4, p=p/p_max)
    elif generator == WAXMAN:
        return nx.waxman_graph(n=num_nodes, alpha=p/p_max, beta=0.1)
    elif generator == POWERLAW:
        return nx.powerlaw_cluster_graph(n=num_nodes, m=3, p=p/(p_max*10))
    else:
        raise ValueError('Generator: {} does not exist'.format(generator))

In [4]:
def create_graphs(paramters, parameter_max, generators):
    graphs = []
    parameters_generators = list(itertools.product(paramters, generators))
    for p, generator in tqdm(parameters_generators):
        graph = create_graph(p, generator, parameter_max)
        graphs.append(OrderedDict([('param', p), 
                                   ('generator', generator), 
                                   ('graph', graph)]))
    return pd.DataFrame(graphs)

In [5]:
graphs_df = create_graphs(PARAMETERS, PARAMETER_MAX, GENERATORS)

In [6]:
graphs_df.head()

,param,generator,graph
0,1,random,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,smallworld,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,1,waxman,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,1,powerlaw,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,2,random,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


#### Coalculate shortest paths and gradients for the networks

In [14]:
def compute_shortest_apth_and_gradients(graphs_df, methods):
    results = []
    for _, (p, generator, graph) in tqdm(list(graphs_df.iterrows())):
        shortest_paths = nx.shortest_path(graph)
        decorated_graph = neg.get_graph_with_energy_data(graph, METHODS, copy=False)
        for source, paths in shortest_paths.items():
            for target, path in paths.items():
                length = len(path) - 1
                if length > 0:
                    for method in methods:
                        gradient = decorated_graph.get_gradient(source, target, method)
                        path_energy = decorated_graph.get_path_energy(path, method)
                        results.append(OrderedDict([('param', p), 
                                                    ('generator', generator), 
                                                    ('method', method), 
                                                    ('source', source), 
                                                    ('target', target), 
                                                    ('length', length), 
                                                    ('gradient', gradient), 
                                                    ('path_energy', path_energy/len(path))]))
    return pd.DataFrame(results)

In [15]:
paths_gradients_df = compute_shortest_apth_and_gradients(graphs_df, METHODS)

In [16]:
paths_gradients_df.head()

,param,generator,method,source,target,length,gradient,path_energy
0,1,random,randic,0,38,1,0.000000,2.000000
1,1,random,laplacian,0,38,1,6.666667,5.333333
2,1,random,graph,0,38,1,2.472136,3.236068
3,1,random,randic,0,17,2,0.000000,2.000000
4,1,random,laplacian,0,17,2,12.444444,8.370370


#### Calculate correlations between shortest path lengths and energy gradients

In [180]:
corr_df = paths_gradients_df\
                .query('target > source')\
                .assign(abs_gradient=np.abs(paths_gradients_df.gradient))\
                .loc[:, ['method', 'generator', 'param', 'length', 'gradient']]\
                .groupby(['method', 'generator', 'param'])\
                .corr(method='pearson')

idx = pd.IndexSlice
corr_df.loc[idx[:, :, :, 'gradient'], 'length'].to_frame().sort_values('length', ascending=False).head()

length
method    generator param                   
randic    powerlaw  3     gradient  0.338101
graph     powerlaw  3     gradient  0.336439
laplacian powerlaw  3     gradient  0.335327
                    2     gradient  0.325801
graph     powerlaw  2     gradient  0.316937

#### Calculate it again using custom function

In [182]:
def compute_correlations(parameters, generators, methods, paths_gradients_df):
    correlations = []
    for p, generator, method in tqdm(list(itertools.product(parameters, generators, methods))):
        sub_df = paths_gradients_df.query("param=='{}' and generator=='{}' and method=='{}'".format(p, generator, method))
        #leave out negative equivalents
        sub_df = sub_df.query("target > source")
        sub_df = sub_df.assign(abs_gradient=np.abs(sub_df.gradient))
        x = list(sub_df.length)
        y = list(sub_df.abs_gradient)
        corr = stats.pearsonr(x, y)
        correlations.append(OrderedDict([('p', p), ('generator', generator), ('method', method), ('corr', corr)]))
    return pd.DataFrame(correlations)

In [184]:
corr_df2 = compute_correlations(PARAMETERS, GENERATORS, METHODS, paths_gradients_df)
corr_df2.sort_values('corr', ascending=True).head()

,p,generator,method,corr
34,3,powerlaw,laplacian,"(-0.354992297403, 6.01383512834e-147)"
33,3,powerlaw,randic,"(-0.35492850351, 6.83760952264e-147)"
22,2,powerlaw,laplacian,"(-0.353338379872, 1.66126846999e-145)"
35,3,powerlaw,graph,"(-0.35191067935, 2.86851749827e-144)"
10,1,powerlaw,laplacian,"(-0.351208813206, 1.15752764371e-143)"


### For empirical networks

In [6]:
datasets = nu.create_datasets('konect.cc').filter(min_size=50, max_size=400, max_density=0.3)

In [7]:
datasets.to_df().head()

,name,category,num_nodes,num_edges,tsv_url
11,iceland,Human contact network,75,114,http://konect.cc/files/download.tsv.iceland.ta...
15,dolphins,Animal network,62,159,http://konect.cc/files/download.tsv.dolphins.t...
17,brunson_revolution,Affiliation network,141,160,http://konect.cc/files/download.tsv.brunson_re...
18,edit-gnwikibooks,Authorship network,68,62,http://konect.cc/files/download.tsv.edit-gnwik...
19,edit-rmwikibooks,Authorship network,70,67,http://konect.cc/files/download.tsv.edit-rmwik...


In [8]:
networks = datasets.download_and_build_networks('data/')

100% [..............................................................................] 14735 / 14735

In [9]:
len(networks)

99

In [10]:
results = []
for row, g in tqdm(list(zip(datasets.to_df().iterrows(), networks))):
    data = row[1]
    shortest_path_lengths = nx.all_pairs_shortest_path_length(g)
    decorated_graph = neg.get_graph_with_energy_data(g, methods, copy=False)
    for method in methods:
        x = []
        y = []
        for lengths in shortest_path_lengths:
            source = lengths[0]
            for target, length in lengths[1].items():
                if target > source:
                    gradient = np.abs(decorated_graph.get_gradient(source, target, method))
                    x.append(gradient)
                    y.append(length)
        if len(x) > 0:
            corr = np.corrcoef(x, y)[0][1]
            pearson = sp.stats.pearsonr(x, y)
            spearman = sp.stats.spearmanr(x, y)
            kendall = sp.stats.kendalltau(x, y)
            results.append((data['name'], data['category'], method, corr, pearson, spearman, kendall))

100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [04:18<00:00,  2.61s/it]


In [11]:
name, category, method, corr, pearson, spearman, kendall = list(zip(*results))

In [12]:
empirical_corrs_df = pd.DataFrame({'name': name, 'category': category, 'method': method,
                                   'corr': corr, 'pearson': pearson, 'spearman': spearman, 'kendall': kendall})

In [13]:
empirical_corrs_df.sort_values('corr')

,category,corr,kendall,method,name,pearson,spearman
3,Authorship network,-0.523494,"(-0.566629023334, 0.0)",graph,edit-gnwikibooks,"(-0.523493725325, 2.32082396477e-319)","(-0.669582715545, 0.0)"
26,Authorship network,-0.512662,"(-0.501487699686, 0.0)",graph,edit-nahwikibooks,"(-0.512662251746, 0.0)","(-0.595273375242, 0.0)"
5,Authorship network,-0.510824,"(-0.488994938551, 2.54880028662e-261)",graph,edit-angwikisource,"(-0.510824348415, 3.20068923876e-220)","(-0.596267440093, 5.39737074143e-319)"
9,Authorship network,-0.507760,"(-0.555734015272, 0.0)",graph,edit-biwikibooks,"(-0.507760074235, 0.0)","(-0.660094532888, 0.0)"
12,Authorship network,-0.506337,"(-0.630421687091, 0.0)",graph,edit-mhwiktionary,"(-0.506337305231, 0.0)","(-0.737882001876, 0.0)"
7,Authorship network,-0.500790,"(-0.609983350428, 0.0)",graph,edit-bmwikibooks,"(-0.500790475629, 0.0)","(-0.717311389651, 0.0)"
20,Authorship network,-0.499985,"(-0.559503646696, 0.0)",graph,edit-kswikibooks,"(-0.499985001127, 0.0)","(-0.655207885762, 0.0)"
37,Authorship network,-0.495902,"(-0.602297802114, 0.0)",graph,edit-xhwiktionary,"(-0.495901584223, 0.0)","(-0.703925647899, 0.0)"
4,Authorship network,-0.495204,"(-0.549484190994, 0.0)",graph,edit-rmwikibooks,"(-0.495204129713, 1.19434295958e-304)","(-0.648580667898, 0.0)"
8,Authorship network,-0.484524,"(-0.584069484953, 0.0)",graph,edit-bmwikiquote,"(-0.48452419153, 0.0)","(-0.695063154459, 0.0)"
